# Factor Investing Backtest

KOSPI200 유니버스에서 모멘텀 팩터 롱숏 전략 백테스트

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(0, '..')

import pandas as pd
import matplotlib.pyplot as plt

# 모듈 임포트
from src.data.kospi200 import get_kospi200_tickers
from src.data.price import fetch_price_data, calculate_returns
from src.factors.momentum import calculate_momentum_12_1
from src.portfolio.backtest import run_backtest, run_quantile_backtest, analyze_backtest_results
from src.analytics.metrics import generate_performance_summary
from src.analytics.visualization import plot_cumulative_returns, plot_drawdown, plot_quantile_returns

print('모듈 로드 완료!')

## 1. 데이터 로드

In [ ]:
# KOSPI200 구성종목
tickers = get_kospi200_tickers('20241201')
print(f'KOSPI200 종목 수: {len(tickers)}')

# 가격 데이터 (2015-2024, 10년)
prices = fetch_price_data(tickers, '2015-01-01', '2024-12-01', use_cache=True)
print(f'가격 데이터: {prices.shape}')
print(f'기간: {prices.index[0].date()} ~ {prices.index[-1].date()}')

## 2. 모멘텀 팩터 정의

In [ ]:
def momentum_factor(prices, date):
    """
    12-1 모멘텀 팩터 계산
    - 12개월 누적 수익률
    - 최근 1개월 스킵 (단기 반전 효과 회피)
    """
    monthly_returns = calculate_returns(prices, period='monthly')
    return calculate_momentum_12_1(monthly_returns, date)

## 3. 백테스트 실행

In [ ]:
# 롱숏 백테스트 (2016-2024)
results = run_backtest(
    prices=prices,
    factor_func=momentum_factor,
    start_date='2016-01-01',
    end_date='2024-11-30',
    rebalance_freq='monthly',
    n_quantiles=5,
    transaction_cost_bps=10
)

print(f'백테스트 기간: {len(results["returns"])} 개월')

## 4. 성과 분석

In [ ]:
# 성과 요약
summary = analyze_backtest_results(results)
print(summary)

In [ ]:
# 누적 수익률 차트
fig = plot_cumulative_returns({'Momentum L/S': results['returns']})
plt.show()

In [ ]:
# 드로다운 차트
fig = plot_drawdown(results['nav'])
plt.show()

## 5. 퀸타일 분석

In [ ]:
# 퀸타일별 수익률
quantile_returns = run_quantile_backtest(
    prices=prices,
    factor_func=momentum_factor,
    start_date='2016-01-01',
    end_date='2024-11-30',
    n_quantiles=5
)

# 퀸타일별 누적 수익률
cumulative = (1 + quantile_returns).prod() - 1
print('퀸타일별 누적 수익률:')
print(cumulative)

In [ ]:
# 퀸타일 차트
fig = plot_quantile_returns(quantile_returns, title='Momentum Factor Quintile Returns')
plt.show()

## 6. 결론

모멘텀 팩터가 KOSPI200에서 유효한지 확인:
- Q5 (고모멘텀) > Q1 (저모멘텀) 이면 팩터 유효
- 롱숏 전략 Sharpe > 0.5 이면 실용적